## Receivable/Invoice

In [1]:
from absbox import API

localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.6, server:0.26.5

In [2]:
invoice = ["Invoice"
            ,{"start":"2024-04-01","originBalance":2000
                ,"originAdvance":1500,"dueDate":"2024-06-01"}
            ,{"status":"Current"}]

In [4]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice]
                     ,poolAssump=("Pool"
                                    ,("Receivable", None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

list(r[0].to_records())

[('2024-04-01', 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 ('2024-06-01', 0, 0, 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

### Default

In [4]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice]
                     ,poolAssump=("Pool",("Receivable"
                                          ,"DefaultAtEnd"
                                          ,None
                                          ,None)
                                        ,None
                                        ,None)
                     ,read=True)

list(r[0].to_records())

[('2024-04-01', 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 ('2024-06-01', 0, 0, 0, 0, 2000, 0, 2000, 0, 0, 0, 2000, 0, 2000)]

In [5]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice]
                     ,poolAssump=("Pool",("Receivable"
                                          ,{'CDR':0.01}
                                          ,None
                                          ,None)
                                        ,None
                                        ,None)
                     ,read=True)


list(r[0].to_records())

[('2024-04-01', 2000, 0, 0., 0, 0., 0, 0., 0, 0, 0, 0., 0, 0.),
 ('2024-06-01', 0, 0, 1996.65, 0, 3.35, 0, 3.35, 0, 0, 0, 3.35, 0, 3.35)]

### Recovery

In [6]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice]
                     ,poolAssump=("Pool",("Receivable"
                                          ,{'CDR':0.01}
                                          ,{"Rate":0.5,"ByDays":[(10,0.5),(20,0.5)]}
                                          ,None)
                                        ,None
                                        ,None)
                     ,read=True)


list(r[0].to_records())

RuntimeError: failed to match {'Rate': 0.5, 'ByDays': [(10, 0.5), (20, 0.5)]}

In [7]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice]
                     ,poolAssump=("Pool",("Receivable"
                                          ,"DefaultAtEnd"
                                          ,{"Rate":0.5,"ByDays":[(10,0.5),(20,0.5)]}
                                          ,None)
                                        ,None
                                        ,None)
                     ,read=True)
list(r[0].to_records())

RuntimeError: failed to match {'Rate': 0.5, 'ByDays': [(10, 0.5), (20, 0.5)]}

### Fee

In [8]:
invoice0 = ["Invoice"
             ,{"start":"2024-04-01","originBalance":2000
              ,"originAdvance":1500,"dueDate":"2024-06-01"
              ,"feeType":("Fixed",150)}
             ,{"status":"Current"}]

In [9]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice0]
                     ,poolAssump=None
                     ,read=True)

list(r[0].to_records())

[('2024-04-01', 2000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 ('2024-06-01', 0, 0, 1850, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [10]:
r = localAPI.runAsset("2024-01-02"
                     ,[invoice0]
                     ,poolAssump=("Pool",("Receivable"
                                          ,{'CDR':0.01}
                                          ,{"Rate":0.5,"ByDays":[(10,0.5),(20,0.5)]}
                                          ,None)
                                        ,None
                                        ,None)
                     ,read=True)

list(r[0].to_records())

[('2024-04-01', 2000, 0, 0., 0., 0., 0., 0., 0, 0, 0, 0., 0., 0.),
 ('2024-06-01', 0, 0, 1846.91, 149.74, 3.35, 0., 0., 0, 0, 0, 3.35, 0., 0.),
 ('2024-06-11', 0, 0, 0., 0., 0., 0.83, 0., 0, 0, 0, 3.35, 0.83, 0.),
 ('2024-06-21', 0, 0, 0., 0., 0., 0.83, 1.68, 0, 0, 0, 3.35, 1.66, 1.68)]